In [1]:


import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load datasets
train_df = pd.read_csv('train_1.csv')
test_df = pd.read_csv('test_1.csv')

In [2]:
train_df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Brown,None reported,Yes,29000
54269,54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,6500
54270,54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,18950
54271,54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,Beige,Brown,None reported,Yes,194965


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [4]:
# Separate features and target
X = train_df.drop(['price', 'id'], axis=1)
y = train_df['price']

# Identify categorical and numerical columns
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Initialize dictionaries to store the LabelEncoders
label_encoders = {}

# Encode categorical variables
for col in cat_cols:
    le = LabelEncoder()
    # Fit on both train and test data to include all possible categories
    le.fit(pd.concat([X[col], test_df[col]]))
    X[col] = le.transform(X[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

# Scale numerical features
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])
   


In [5]:
!pip install lightgbm

In [6]:

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train LightGBM model
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f' RMSE: {rmse}')

 RMSE: 58323.32657020602


In [7]:
# Hyperparameter tuning using Grid Search

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# Predict on the test set
best_model = grid_search.best_params_
print(best_model)



{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}


In [8]:
# Create a new model with the best parameters
best_runner = XGBRegressor(**best_model, random_state=42)

# Fit the model on the full training data
best_runner.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_runner.predict(X_val)


# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'Best RMSE on validation set: {rmse}')

Best RMSE on validation set: 48212.365028621825


In [9]:
test_df_features = test_df.drop(columns=['id'])

test_preds = best_runner.predict(test_df_features)

# Prepare submission
submission = pd.DataFrame({'id': test_df['id'], 'price': test_preds})
submission

,id,price
0,54273,25173.673828
1,54274,18553.216797
2,54275,26176.167969
3,54276,64388.707031
4,54277,36408.320312
...,...,...
36178,90451,75741.125000
36179,90452,19692.396484
36180,90453,13469.843750
36181,90454,52722.078125


In [11]:
local_path = 'C:/Users/User/Downloads/submission.csv'  # Adjust this path
submission.to_csv(local_path, index=False)